In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("241211_03_SparkSQL_UDF").getOrCreate()

24/12/11 16:10:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/11 16:10:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/11 16:10:37 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
datas = [
    ("A", "2022-04-16", 31200),
    ("B", "2022-04-17", 41200),
    ("C", "2022-04-11", 31500),
    ("D", "2022-04-12", 21500),
    ("E", "2022-04-13", 51000)
]
columns = ["product", "date", "price"]

In [3]:
df = spark.createDataFrame(data=datas, schema=columns)
df.show()

+-------+----------+-----+
|product|      date|price|
+-------+----------+-----+
|      A|2022-04-16|31200|
|      B|2022-04-17|41200|
|      C|2022-04-11|31500|
|      D|2022-04-12|21500|
|      E|2022-04-13|51000|
+-------+----------+-----+



In [ ]:
#UDF

In [4]:
from pyspark.sql.types import LongType

def squared(n):
    return n*n

In [8]:
spark.udf.register('squared', squared,LongType())

<function __main__.squared(n)>

In [6]:
df.createOrReplaceTempView('product')

In [7]:
spark.sql("select * from product").show()

+-------+----------+-----+
|product|      date|price|
+-------+----------+-----+
|      A|2022-04-16|31200|
|      B|2022-04-17|41200|
|      C|2022-04-11|31500|
|      D|2022-04-12|21500|
|      E|2022-04-13|51000|
+-------+----------+-----+



In [10]:
spark.sql("select price, squared(price) from product").show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|31200|     973440000|
|41200|    1697440000|
|31500|     992250000|
|21500|     462250000|
|51000|    2601000000|
+-----+--------------+



In [17]:
def read_number(n) :
    units = ["", "십", "백", "천", "만"]
    nums = '일이삼사오육칠팔구'
    result = []
    i=0
    while n>0:
        n,r = divmod(n,10)
        if r>0:
            result.append(nums[r-1]+units[i])
        i += 1
    return "".join(reversed(result))

In [18]:
read_number(123)

'일백이십삼'

In [19]:
spark.udf.register('read_number', read_number)

<function __main__.read_number(n)>

In [21]:
spark.sql('select price, read_number(price) from product').show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|31200|      삼만일천이백|
|41200|      사만일천이백|
|31500|      삼만일천오백|
|21500|      이만일천오백|
|51000|          오만일천|
+-----+------------------+



In [22]:
spark.stop()